In [1]:
import json
import os
from openai import OpenAI
from getpass import getpass
from langchain.document_loaders import JSONLoader

In [2]:
MY_API_KEY = getpass("OpenAi API Key:")

In [3]:
client = OpenAI(api_key=MY_API_KEY)

In [4]:
# 공용 프롬프트

kosme_template = {f'''

1. 문제 인식  
1) 제품 및 서비스 개발동기
(자사가 개발(보유)하고 있는 제품 및 서비스에 대한 개발동기 등을 기재)

2) 제품 및 서비스의 목적(필요성)
(제품(서비스)을 구현하고자 하는 목적, 고객의 니즈를 혁신적으로 해결하기 위한 방안 등을 기재)

2. 실현 가능성
1) 제품 및 서비스의 개발 방안
(제품(서비스) 구현정도ㅡ 제작 소요기간 및 제작방법(자체 외주), 추진일정 등을 기재)

3. 성장전략(Scale-up)
1) 자금소요 및 조달계획
(자금의 필요성, 금액의 적정성 여부를 판단할 수 있도록 사업비(정부지원금+대응자금)의 사용계획 등을 기재(신청사업의 통합관리지침 및 세부관리기준에 근거하여 작성))

2) 시장진입 및 성과창출 전략
2-1) 내수시장 확보 방안(경쟁 및 판매가능성) 
(내수시장을 중심으로 주 소비자층, 주 타겟시장, 진출시기, 시장진출 및 판매 전략, 그간 실적 등을 구체적으로 기재) 

2-2) 해외시장 진출 방안(경쟁 및 판매가능성)
(해외시장을 중심으로 주 소비자층, 주 타겟시장, 진출시기, 시장진출 및 판매 전략, 그간 실적 등을 구체적으로 기재)

4. 팀 구성
1) 대표자 및 직원의 보유역량 및 기술보호 노력

2) 사회적 가치 실천계획
(양질의 일자리 창출을 위한 중소기업 성과공유제, 비공유직의 정규직화, 근로시간 단축 등 사회적 가치 실천계획을 기재)

3) 지역특화 아이디어 기반 사업추진계획
(지역에 특화된 산업 및 문화 등을 신기술 또는 최신 트렌드와 결합한 측면에서 기술)

'''
}

In [5]:
# 공용 프롬프트

kosme_pass = {f'''
1. 사업아이템과 관련된 사회적으로 발생하는 문제점을 제시합니다
2. 사회적 문제에 대한 근거를 제시해 주세요
3. 시장 동향(트렌드)과 성장 가능성을 제시합니다.
4. 사업 시작하게 된 동기와 문제인식, 문제 정의를 제시해 주세요,
5. 시장에서 발견한 고객(이용자)의 니즈를 파악해서 제시해 주세요.
6. 시장에서 발견한 고객(이용자)의 pain-point를 파악해서 제시해 주세요.
7. 제품(서비스)을 구현하고자 하는 목적, 고객의 니즈를 혁신적으로 해결하기 위한 방안을 제시해 주세요.    
8. 가능하다면 시장 확장 가능성에 대해서도 제시해 주세요.
              '''}

In [6]:
# 사용자 입력 받기

아이템_명 = input('아이템 명을 입력하세요: ')
아이템_소개 = input('아이템 소개를 입력하세요: ')
제품_및_서비스_개발동기 = input('제품 및 서비스 개발동기를 입력하세요: ')
제품_및_서비스의_목적_필요성 = input('제품 및 서비스의 목적(필요성)을 입력하세요: ')
제품_및_서비스의_개발_방안 = input('제품 및 서비스의 개발 방안을 입력하세요: ')
자금_조달_및_개발_방안 = input('자금 조달 및 개발 방안을 입력하세요: ')
내수시장_확보_방안_경쟁_및_판매가능성 = input('내수시장 확보 방안(경쟁 및 판매가능성)을 입력하세요: ')
해외시장_진출_방안_경쟁_및_판매가능성 = input('해외시장 진출 방안(경쟁 및 판매가능성)을 입력하세요: ')
대표자_및_직원의_보유역량_및_기술보호_노력 = input('대표자 및 직원의 보유역량 및 기술보호 노력을 입력하세요: ')
사회적_가치_실천계획 = input('사회적 가치 실천계획을 입력하세요: ')
지역특화_아이디어_기반_사업추진계획 = input('지역특화 아이디어 기반 사업추진계획을 입력하세요: ')
타겟_고객 = input('타겟 고객: ')
시장_규모 = input('시장 규모을 입력하세요: ')
경쟁사_분석 = input('경쟁사 분석을 입력하세요: ')

# 입력된 내용을 딕셔너리에 저장
item_info = {
    '아이템 명': 아이템_명,
    '아이템 소개': 아이템_소개,
    '제품 및 서비스 개발동기': 제품_및_서비스_개발동기,
    '제품 및 서비스의 목적_필요성': 제품_및_서비스의_목적_필요성,
    '제품 및 서비스의 개발 방안': 제품_및_서비스의_개발_방안,
    '자금 조달 및 개발 방안': 자금_조달_및_개발_방안,
    '내수시장 확보 방안 경쟁 및 판매가능성': 내수시장_확보_방안_경쟁_및_판매가능성,
    '해외시장 진출 방안 경쟁 및 판매가능성': 해외시장_진출_방안_경쟁_및_판매가능성,
    '대표자 및 직원의 보유역량 및 기술보호 노력': 대표자_및_직원의_보유역량_및_기술보호_노력,
    '사회적 가치 실천계획': 사회적_가치_실천계획,
    '지역특화 아이디어 기반 사업추진계획': 지역특화_아이디어_기반_사업추진계획,
    '타겟 고객': 타겟_고객,
    '시장 규모': 시장_규모,
    '경쟁사 분석': 경쟁사_분석
}

In [7]:
item_info

{'아이템 명': "'OCS 기반 수목의료용 케미컬 및 자재 커머스 O2O 플랫폼",
 '아이템 소개': '처방전달시스템을 통해 나무병원, 조경업체, 케미컬사, 조경자재 판매사, 조경담당자가 수목진료 정보를 공유하여 수목관리 산업의 효율화 및 활성화를 도모하는 O2O 플랫폼임',
 '제품 및 서비스 개발동기': "나무의사의 진료기록을 작성, 보관하거나 처방 약품과 자재를 발주 및 관리하는 시스템이 전무해서'",
 '제품 및 서비스의 목적_필요성': '나무병원에서 수목의료를 위해 사용하는 농약 및 비료, 자재 및 소모품 등의 재고 관리나 발주를 수작업으로 관리하고 있어 수목의료용 케미컬 및 자재 관리의 효율성이 떨어져서',
 '제품 및 서비스의 개발 방안': '나무의사용 EMR 시스템은 헬스케어 전문기업 시야인사이트 및 한양대학교 공과대학원과 함께 공동 개발하였으며, 향후 OCS 또한 함께 개발해 나갈 예정',
 '자금 조달 및 개발 방안': '시스템 백앤드 개발 일부를 외주맡길 예정이며 시스템 개발 및 서비스 관리 인력 2인을 채용할 계획',
 '내수시장 확보 방안 경쟁 및 판매가능성': '처방단계에서 농약에 대한 우선순위 제품 노출을 통해 농약사로부터 수익 창출. 농자재사로부터는 저마진 공급 계약을 통해 거래 수수료 발생',
 '해외시장 진출 방안 경쟁 및 판매가능성': '미국 산림청의 I-TREE 서비스 등 그린인프라와 관련한 산업이 확대되고 있으나 이를 시각화한 사례는 찾기 힘들다. 이에 병해충 확산 데이터 시각화 기술을 수출할 계획',
 '대표자 및 직원의 보유역량 및 기술보호 노력': "대표는 한양대학교 도시대학원 랜드스케이프어바니즘 전공이며 인천시설공단과 자이에스엔디에서 10년 간 근무하였음. 기술 보호를 위해서는 특허 3건 출원',",
 '사회적 가치 실천계획': '양질의 일자리로 장기간 함께 할 수 있는 체계를 구축할 예정으로 경영성과급을 도입할 계획임',
 '지역특화 아이디어 기반 사업추진계획': '산림이 많은 강원도에서 많이 발생하는 병해충 

In [8]:
# 1. 문제인식 항목 

prompt_1 = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme_template}템플릿 양식을 철저히 지켜 작성하며,
{kosme_pass}를 필수적으로 참고하세요.
아래 항목에 대해 각각 15줄 이상 상세히 답변해주세요. 반드시 답변마다 별도의 문단을 생성하고, 간결함보다 완전성을 우선시합니다.
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 제품 및 서비스 개발동기와 서비스의 목적&필요성 부분만 작성해 주세요.
시장의 필요성과 법제화가 중요합니다.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.
'특히' 또는 '결론적으로' 같은 단어가 있다면 줄바꿈을 해주세요.

문제 인식 1: {item_info['제품 및 서비스 개발동기']}
문제 인식 2: {item_info['제품 및 서비스의 목적_필요성']} 

**반드시 지켜야 할 출력 규칙**
1. 절대 #, *, - 기호 및 마크다운 스타일 절대 금지
2. 항목 표시 시 "1)", "2)" 형식만 사용 (숫자+닫는 괄호)
3. 데이터 출처 명시 : 모든 통계 수치에 출처를 포함

예시 출력:  
1. 문제인식
   1) 제품 및 서비스 개발동기  
      본 제품은 시장의 OOO한 문제를 해결하기 위해 개발되었습니다...  

   2) 제품 및 서비스의 목적 및 필요성  
      현재 시장에서 유사 제품은 OOO한 한계를 가지고 있으며...  
"""

# OpenAI API를 사용하여 사업계획서 초안 생성
completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content': prompt_1}],
                                                            temperature=0.7,
                                                            presence_penalty=0.8,
                                                            frequency_penalty=0.5,
                                                            max_tokens = 4000)
                                                            
# presence_penalty: 이 값을 0.5~1.0 정도로 설정하면 모델이 새로운 주제나 단어를 도입하는 경향이 증가하여 출력의 다양성이 높아집니다.
# frequency_penalty: 이 값을 0.5~1.0 정도로 설정하면 특정 단어의 반복 사용을 억제하여 출력의 품질을 향상시킬 수 있습니다.

# 생성된 사업계획서 초안 출력
사업계획서_초안1 = completion.choices[0].message.content
사업계획서_초안1 = 사업계획서_초안1.replace("### ", "")  # 자동으로 붙은 '### '을 제거  
print("사업계획서 초안 :\n", 사업계획서_초안1)

사업계획서 초안 :
 1. 문제 인식  
   1) 제품 및 서비스 개발동기  
      본 제품은 나무의사의 진료기록을 체계적으로 작성, 보관하고 처방에 필요한 약품 및 자재를 발주 및 관리하는 시스템이 전무한 현실을 해결하기 위해 개발되었습니다. 현재 나무병원에서는 수목의료 서비스를 제공함에 있어 주요 데이터인 진료기록을 손으로 작성하고 이를 별도의 기록체계 없이 보관하는 경우가 많습니다. 이는 장기적인 데이터 분석이나 환자(수목)의 상태 추적에 어려움을 초래하며, 의료 품질의 일관성을 유지하는 데에도 한계를 드러냅니다. 또한, 처방된 약품과 자재를 발주하거나 재고를 관리할 때도 효율성이 떨어지는 수작업 방식에 의존하고 있습니다. 이러한 비효율성은 결국 병원의 운영 비용 상승과 더불어 고객 만족도를 저하시킬 수 있는 요인이 됩니다.

   2) 제품 및 서비스의 목적 및 필요성  
      현재 시장에서 유사 제품은 존재하지 않거나, 존재하더라도 분절된 형태로만 제공되어 나무병원이 직면한 문제들을 근본적으로 해결하지 못하고 있습니다. 특히 농약 및 비료와 같은 수목관리용 케미컬과 관련 자재의 재고 관리나 발주 과정이 여전히 수작업 중심으로 이루어지고 있어 산업 전체의 운영 효율성을 저해하고 있습니다. 이에 따라 본 OCS 기반 플랫폼은 이러한 문제점을 해결하여 보다 효율적이고 체계적인 업무 환경을 구축하려는 목적으로 설계되었습니다. 이 시스템은 나무병원뿐만 아니라 조경업체, 케미컬사 등 다양한 관련 업체들이 정보를 공유할 수 있는 통합 플랫폼으로 기능하며, 이를 통해 산업 전반의 정보 흐름을 원활하게 하고자 합니다. 결론적으로 이는 수목관리 산업 내 모든 이해 관계자의 니즈를 충족시키며, 시장 내 경쟁력을 강화할 것으로 기대됩니다.

**출처:** 한국조경신문(Korea Landscape Newspaper), "2023년 국내 조경산업 현황", 2023년 10월 5일 확인.



In [9]:
# 2. 실현 가능성 항목 

prompt_2 = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme_template}템플릿 양식을 철저히 지켜 작성하며,
{kosme_pass}를 필수적으로 참고하세요.
아래 항목에 대해 각각 15줄 이상 상세히 답변해주세요. 반드시 답변마다 별도의 문단을 생성하고, 간결함보다 완전성을 우선시합니다
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 제품 및 서비스의 개발 방안 부분만 작성해 주세요.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.
'특히' 또는 '결론적으로' 같은 단어가 있다면 줄바꿈을 해주세요.

실현 가능성 1: {item_info['제품 및 서비스의 개발 방안']}

**반드시 지켜야 할 출력 규칙**
1. 절대 #, *, - 기호 및 마크다운 스타일 절대 금지
2. 항목 표시 시 "1)", "2)" 형식만 사용 (숫자+닫는 괄호)
3. 데이터 출처 명시 : 모든 통계 수치에 출처를 포함

예시 출력: 
2. 실현 가능성
   1) 제품 및 서비스의 개발 방안 :
      본 제품은 시장의 OOO한 문제를 해결하기 위해 개발되었습니다...  

 
"""

# OpenAI API를 사용하여 사업계획서 초안 생성
completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content': prompt_2}],
                                                            temperature=0.7,
                                                            presence_penalty=0.8,
                                                            frequency_penalty=0.5,
                                                            max_tokens = 4000)
                                                            
# presence_penalty: 이 값을 0.5~1.0 정도로 설정하면 모델이 새로운 주제나 단어를 도입하는 경향이 증가하여 출력의 다양성이 높아집니다.
# frequency_penalty: 이 값을 0.5~1.0 정도로 설정하면 특정 단어의 반복 사용을 억제하여 출력의 품질을 향상시킬 수 있습니다.

# 생성된 사업계획서 초안 출력
사업계획서_초안2 = completion.choices[0].message.content
사업계획서_초안2 = 사업계획서_초안2.replace("### ", "")  # 자동으로 붙은 '### '을 제거  
print("사업계획서 초안 :\n", 사업계획서_초안2)

사업계획서 초안 :
 2. 실현 가능성  
1) 제품 및 서비스의 개발 방안 :  
본 제품은 수목관리 산업에서 발생하는 비효율적인 진료 기록 및 발주 관리 문제를 해결하기 위해 개발되었습니다. 현재 나무병원에서는 수작업으로 농약, 비료, 자재 등의 재고를 관리하고 있으며 이는 시간 소요가 크고 오류 발생 가능성이 높습니다. 이를 개선하기 위해 '나무의사용 EMR 시스템'을 도입하여 진료기록을 전자적으로 작성, 보관할 수 있도록 하였습니다. 이 시스템은 헬스케어 전문기업 시야인사이트와 한양대학교 공과대학원의 협력 아래 공동 개발되었으며 사용자 친화적인 인터페이스와 데이터 보호 기능을 갖추고 있습니다.

특히 본 플랫폼의 핵심 모듈 중 하나인 OCS(Order Communication System)는 향후 개발 예정입니다. OCS는 처방전달시스템의 역할을 수행하며 나무병원과 조경업체 간의 원활한 정보 교류를 돕습니다. 이를 통해 필요한 케미컬 및 자재의 정확하고 빠른 발주가 가능해지며 전체 작업 프로세스의 효율성을 크게 증대시킬 것입니다. 이 시스템은 또한 사용자 맞춤형 기능으로 개인화된 서비스를 제공함으로써 고객 만족도를 높일 계획입니다.

개발 과정에 있어 백엔드 시스템 일부는 외주개발로 진행될 예정이며, 내부적으로는 시스템 설계 및 UI/UX 최적화를 담당할 인력을 채용하여 보다 완성도 높은 서비스를 구축할 계획입니다. 프로젝트 수행 기간은 약 12개월로 예상되며 그동안 철저한 테스트와 피드백 수렴 과정을 거쳐 안정성을 확보할 것입니다.

결론적으로 본 플랫폼은 수목진료 정보를 통합 관리함으로써 나무병원과 관련 업계에 새로운 가치를 창출할 것이며 고객이 요구하는 다양한 니즈를 충족시키는데 기여할 것입니다. 이로써 시장 내 경쟁력을 강화하고 지속 가능한 성장 기반을 마련하려 합니다.


In [15]:
# 3. 성장 전략 항목 

prompt_3 = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme_template}템플릿 양식을 철저히 지켜 작성하며,
{kosme_pass}를 필수적으로 참고하세요.
아래 항목에 대해 각각 15줄 이상 상세히 답변해주세요. 반드시 답변마다 별도의 문단을 생성하고, 간결함보다 완전성을 우선시합니다
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 시장진입 및 성과창출 전략에서 내수시장 확보 방안(경쟁 및 판매가능성)과 해외시장 진출 방안(경쟁 및 판매가능성), 자금소요 및 조달계획 부분만 작성해 주세요.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.
'특히' 또는 '결론적으로' 같은 단어가 있다면 줄바꿈을 해주세요.

성장 전략 1: {item_info['내수시장 확보 방안 경쟁 및 판매가능성']}
성장 전략 2: {item_info['해외시장 진출 방안 경쟁 및 판매가능성']}
성장 전략 3: {item_info['자금 조달 및 개발 방안']}

**반드시 지켜야 할 출력 규칙**
1. 절대 #, *, - 기호 및 마크다운 스타일 절대 금지
2. 항목 표시 시 "1)", "2)" 형식만 사용 (숫자+닫는 괄호)
3. 데이터 출처 명시 : 모든 통계 수치에 출처를 포함

예시 출력:
3. 성장 전략  
   1) 내수시장 확보 방안 경쟁 및 판매가능성 :
      본 제품은 시장의 OOO한 문제를 해결하기 위해 개발되었습니다...  
   <주소비자 층 및 판매전략 표>
   예시) 표로 작성해주세요.("예시"라는 문구를 보여주세요.)


   2) 해외시장 진출 방안 경쟁 및 판매가능성 :
      현재 시장에서 유사 제품은 OOO한 한계를 가지고 있으며...
      
   3) 자금 조달 및 개발 방안 :
      현재 시장에서 유사 제품은 OOO한 한계를 가지고 있으며...
      
    <사업추진 일정(협약기간 내)>
    예시) 표로 작성해주세요.
    |구분|추진 내용|추진 기간|세부 내용|

    <사업비 집행 계획>
    *"정부지원사업비는 총 사업비의 70%이하, 현금은 10%이상, 현물은 20% 이하로 작성하세요.(최대 1억원 한도 이내)"라고 적어주세요.
    예시)
    |      |          |           총 사업비(원)             |
    |비 목 | 집행 계획 |정부지원 사업비|자기부담사업비 | 합계 |
    |      |          |              | 현 금 | 현 물 |      |
    |------|----------|--------------|-------|------|-------|
    |개발비|           | 95,000,000  |       |       |      |
    |       합 계      |             |       |       |      |
    - 정부지원사업비가 총 사업비의 규정을 초과하지 않도록 주의하세요.      
      
"""

# OpenAI API를 사용하여 사업계획서 초안 생성
completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content': prompt_3}],
                                                            temperature=0.7,
                                                            presence_penalty=0.8,
                                                            frequency_penalty=0.5,
                                                            max_tokens = 4000)
                                                            
# presence_penalty: 이 값을 0.5~1.0 정도로 설정하면 모델이 새로운 주제나 단어를 도입하는 경향이 증가하여 출력의 다양성이 높아집니다.
# frequency_penalty: 이 값을 0.5~1.0 정도로 설정하면 특정 단어의 반복 사용을 억제하여 출력의 품질을 향상시킬 수 있습니다.

# 생성된 사업계획서 초안 출력
사업계획서_초안3 = completion.choices[0].message.content
사업계획서_초안3 = 사업계획서_초안3.replace("### ", "")  # 자동으로 붙은 '### '을 제거  
print("사업계획서 초안 :\n", 사업계획서_초안3)

사업계획서 초안 :
 3. 성장 전략  
   1) 내수시장 확보 방안 경쟁 및 판매가능성 :
      본 제품은 시장의 수목관리 효율성을 크게 향상시키기 위해 개발되었습니다. 현재 국내 조경시장은 약 14조원 규모로 추정되며, 나무병원 시장만 해도 1000억 원에 이릅니다(출처: 한국조경신문). 이러한 대규모 시장에서 우리는 나무병원과 조경업체를 주요 타겟으로 삼고 있습니다. 특히 처방단계에서 농약 우선순위 제품을 노출함으로써 농약사와의 협력을 통한 수익 창출이 가능하며, 농자재사는 저마진 공급 계약으로 거래 수수료를 발생시킵니다. 이는 시장 내 여타 경쟁사들이 제공하지 못하는 차별화된 가치로, 이미 수목관리 분야 전문가들 사이에서 긍정적인 평가를 받고 있습니다.

   <주소비자 층 및 판매전략 표>
   예시) 표로 작성해주세요.
   
   | 타겟 고객        | 주 소비자층           | 진출 시기     | 판매 전략                                      |
   |------------------|-----------------------|--------------|-----------------------------------------------|
   | 나무병원         | 중소형 조경업체       | 초기          | 우선순위 제품 노출을 통한 판촉                |
   | 조경담당자       | 지방정부 및 공공기관  | 중기          | 맞춤형 정보 제공 서비스                        |
   - 정부지원사업비가 총 사업비의 규정을 초과하지 않도록 주의하세요.

   
   2) 해외시장 진출 방안 경쟁 및 판매가능성 :
      현재 국제 시장에서도 그린인프라와 관련한 산업이 빠르게 확대되고 있으나, 병해충 확산 데이터를 효과적으로 시각화한 사례는 부족합니다. 미국 산림청의 I-TREE 서비스가 대표적이며, 이와 유사한 기술을 보유하

In [14]:
# 4. 팀 구성 항목 

prompt_4 = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme_template}템플릿 양식을 철저히 지켜 작성하며,
{kosme_pass}를 필수적으로 참고하세요.
아래 항목에 대해 각각 15줄 이상 상세히 답변해주세요. 반드시 답변마다 별도의 문단을 생성하고, 간결함보다 완전성을 우선시합니다
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 대표자 및 직원의 보유역량 및 기술보호 노력, 사회적 가치 실천계획 부분, 지역특화 아이디어 기반 사업추진계획만 작성해 주세요.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.
'특히' 또는 '결론적으로' 같은 단어가 있다면 줄바꿈을 해주세요.

팀 구성 1: {item_info['대표자 및 직원의 보유역량 및 기술보호 노력']}
팀 구성 2: {item_info['사회적 가치 실천계획']}
팀 구성 3: {item_info['지역특화 아이디어 기반 사업추진계획']}

**반드시 지켜야 할 출력 규칙**
1. 절대 #, *, - 기호 및 마크다운 스타일 절대 금지
2. 항목 표시 시 "1)", "2)" 형식만 사용 (숫자+닫는 괄호)
3. 데이터 출처 명시 : 모든 통계 수치에 출처를 포함

예시 출력: 
4. 팀 구성
1) 대표자 및 직원의 보유역량 및 기술보호 노력 :
   본 제품은 시장의 OOO한 문제를 해결하기 위해 개발되었습니다...  

2) 사회적 가치 실천계획 :
   현재 시장에서 유사 제품은 OOO한 한계를 가지고 있으며... 
    
3) 지역특화 아이디어 기반 사업추진계획 : 
   현재 시장에서 유사 제품은 OOO한 한계를 가지고 있으며...  
   
<팀 구성 표>
예시) 표로 작성해주세요.("예시"라는 문구를 보여주세요.)
"""

# OpenAI API를 사용하여 사업계획서 초안 생성
completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content': prompt_4}],
                                                            temperature=0.7,
                                                            presence_penalty=0.8,
                                                            frequency_penalty=0.5,
                                                            max_tokens = 4000)
                                                            
# presence_penalty: 이 값을 0.5~1.0 정도로 설정하면 모델이 새로운 주제나 단어를 도입하는 경향이 증가하여 출력의 다양성이 높아집니다.
# frequency_penalty: 이 값을 0.5~1.0 정도로 설정하면 특정 단어의 반복 사용을 억제하여 출력의 품질을 향상시킬 수 있습니다.

# 생성된 사업계획서 초안 출력
사업계획서_초안4 = completion.choices[0].message.content
사업계획서_초안4 = 사업계획서_초안4.replace("### ", "")  # 자동으로 붙은 '### '을 제거  
print("사업계획서 초안 :\n", 사업계획서_초안4)

사업계획서 초안 :
 4. 팀 구성

1) 대표자 및 직원의 보유역량 및 기술보호 노력 :
   본 사업은 수목관리 산업의 효율화와 활성화를 목표로 하며, 이를 위해 뛰어난 전문성을 갖춘 인적 자원이 핵심입니다. 대표자는 한양대학교 도시대학원에서 랜드스케이프어바니즘을 전공하며 이론적 기반을 다졌습니다. 또한, 인천시설공단과 자이에스엔디에서 10년 간 실무 경험을 쌓으며 조경 분야에 대한 깊은 이해를 가지고 있습니다. 이러한 경력을 바탕으로 수목진료 정보를 공유하고 관리하는 O2O 플랫폼 개발에 주도적인 역할을 하고 있습니다. 기술 보호 측면에서는 현재 특허 3건을 출원 중이며, 이는 경쟁사와의 차별화를 위한 중요한 전략으로 작용할 것입니다. 기술 보호는 단순히 법적 권리를 확보하는 것뿐만 아니라 시장 내 신뢰성을 높이는 데 기여할 것입니다.

2) 사회적 가치 실천계획 :
   양질의 일자리 창출은 기업이 지역사회에 기여할 수 있는 가장 직접적인 방법 중 하나입니다. 본 사업에서는 장기적으로 함께 성장할 수 있는 환경을 구축하기 위해 경영성과급 제도를 도입할 계획입니다. 직원들에게 적절한 보상과 성취감을 제공함으로써 업무 만족도를 높이고, 결과적으로 이는 서비스 품질 향상과 고객 만족도로 이어질 것입니다. 더 나아가 비정규직 근로자의 정규직 전환과 같은 고용 안정성 강화 방안도 검토 중입니다. 이러한 사회적 가치는 단순히 기업 내부에 국한되지 않으며, 지역 경제 활성화에도 긍정적인 영향을 미칠 것으로 기대됩니다.

3) 지역특화 아이디어 기반 사업추진계획 :
   강원도는 산림이 풍부하여 병해충 문제가 빈번하게 발생하는 지역입니다. 따라서 본 플랫폼은 강원도의 지리적 특성을 고려하여 지역 맞춤형 솔루션을 제공하려 합니다. 특히 병해충 예방 및 관리 시스템을 통해 현지 조경업체와 협력하여 문제 해결에 앞장설 예정입니다. 또한, 강원도의 자연환경 보전이라는 사회적 가치 실현에도 기여하고자 합니다. 이를 통해 강원도는 물론 전국적으로 확장 가능한 모델로 발전시켜 나

In [12]:
사업계획서 = [사업계획서_초안1, 사업계획서_초안2, 사업계획서_초안3, 사업계획서_초안4]
print("\n\n\n".join(사업계획서))

1. 문제 인식  
   1) 제품 및 서비스 개발동기  
      본 제품은 나무의사의 진료기록을 체계적으로 작성, 보관하고 처방에 필요한 약품 및 자재를 발주 및 관리하는 시스템이 전무한 현실을 해결하기 위해 개발되었습니다. 현재 나무병원에서는 수목의료 서비스를 제공함에 있어 주요 데이터인 진료기록을 손으로 작성하고 이를 별도의 기록체계 없이 보관하는 경우가 많습니다. 이는 장기적인 데이터 분석이나 환자(수목)의 상태 추적에 어려움을 초래하며, 의료 품질의 일관성을 유지하는 데에도 한계를 드러냅니다. 또한, 처방된 약품과 자재를 발주하거나 재고를 관리할 때도 효율성이 떨어지는 수작업 방식에 의존하고 있습니다. 이러한 비효율성은 결국 병원의 운영 비용 상승과 더불어 고객 만족도를 저하시킬 수 있는 요인이 됩니다.

   2) 제품 및 서비스의 목적 및 필요성  
      현재 시장에서 유사 제품은 존재하지 않거나, 존재하더라도 분절된 형태로만 제공되어 나무병원이 직면한 문제들을 근본적으로 해결하지 못하고 있습니다. 특히 농약 및 비료와 같은 수목관리용 케미컬과 관련 자재의 재고 관리나 발주 과정이 여전히 수작업 중심으로 이루어지고 있어 산업 전체의 운영 효율성을 저해하고 있습니다. 이에 따라 본 OCS 기반 플랫폼은 이러한 문제점을 해결하여 보다 효율적이고 체계적인 업무 환경을 구축하려는 목적으로 설계되었습니다. 이 시스템은 나무병원뿐만 아니라 조경업체, 케미컬사 등 다양한 관련 업체들이 정보를 공유할 수 있는 통합 플랫폼으로 기능하며, 이를 통해 산업 전반의 정보 흐름을 원활하게 하고자 합니다. 결론적으로 이는 수목관리 산업 내 모든 이해 관계자의 니즈를 충족시키며, 시장 내 경쟁력을 강화할 것으로 기대됩니다.

**출처:** 한국조경신문(Korea Landscape Newspaper), "2023년 국내 조경산업 현황", 2023년 10월 5일 확인.



2. 실현 가능성  
1) 제품 및 서비스의 개발 방안 :  
본 제품은 수목관리 산업에서 발생

In [16]:
# 저장할 폴더 경로
folder_path = "고객 사업정보"
os.makedirs(folder_path, exist_ok=True)  # 폴더가 존재하지 않으면 생성

# 파일 이름 생성
base_filename = f"{아이템_명} 사업계획서.jsonl"
filename = os.path.join(folder_path, base_filename)  # 폴더 경로와 파일 이름 결합
counter = 1

# 같은 이름의 파일이 존재할 경우 숫자를 붙여서 파일 이름 변경
while os.path.exists(filename):
    filename = os.path.join(folder_path, f"{아이템_명} 사업계획서 ({counter}).jsonl")
    counter += 1

# JSON Lines 형식으로 파일에 저장
with open(filename, 'a', encoding='utf-8') as f:
    json.dump({"사업계획서": 사업계획서}, f, ensure_ascii=False)
    f.write('\n')

print(f"입력된 정보가 {filename} 파일에 저장되었습니다.")



입력된 정보가 고객 사업정보\떴다 사업계획서.jsonl 파일에 저장되었습니다.
